Некоторые задания в этой тетрадки были созданы на основе соотвествующей тетрадки курса NLP от Elena Voita.

# Генерация текста: н-граммы

В этой тетрадке мы научимся делать простую модель генерации текста на основе н-грамм и встречаемости в корпусе.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [2]:
random.seed(1)
np.random.seed(1)

Будем пробовать генерировать шутки. Для обучения будем использовать [датасет с постами reddit](https://kaggle.com/datasets/thedevastator/one-million-reddit-jokes).

In [3]:
path = '/content/drive/MyDrive/NLP Course Tutoring/datasets/reddit_jokes.csv'
data = pd.read_csv(path)

In [4]:
data.head()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
0,post,ftbp1i,2qh72,jokes,False,1585785543,https://old.reddit.com/r/Jokes/comments/ftbp1i...,self.jokes,NaN,My corona is covered with foreskin so it is no...,I am soooo glad I'm not circumcised!,2
1,post,ftboup,2qh72,jokes,False,1585785522,https://old.reddit.com/r/Jokes/comments/ftboup...,self.jokes,NaN,It's called Google Sheets.,Did you know Google now has a platform for rec...,9
2,post,ftbopj,2qh72,jokes,False,1585785508,https://old.reddit.com/r/Jokes/comments/ftbopj...,self.jokes,NaN,The vacuum doesn't snore after sex.\r\n\r\n&am...,What is the difference between my wife and my ...,15
3,post,ftbnxh,2qh72,jokes,False,1585785428,https://old.reddit.com/r/Jokes/comments/ftbnxh...,self.jokes,NaN,[removed],My last joke for now.,9
4,post,ftbjpg,2qh72,jokes,False,1585785009,https://old.reddit.com/r/Jokes/comments/ftbjpg...,self.jokes,NaN,[removed],The Nintendo 64 turns 18 this week...,134


Так как наша задача генерации требует только текста, оставим только соответствующий столбец.

In [5]:
columns = ['selftext']
data = data[columns]

## Обработка данных
###1. Чистка датасета
Для начала нужно определиться, есть в данных пропуски, и избавиться от них, если есть.

__Подсказка:__ Часто пропуски они обозначаются как nan, но иногда можно заметить иные способы.

In [1]:
# <YOUR CODE HERE> #

Надо тексты привести к нижнему регистру и убрать пунктуацию.
Кроме этого можно избавиться от совсем коротких шуток, так как скорее всего это просто ответы на фразы. При делении текста на слова, используйте word_tokenize.

In [22]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Надо тексты привести к нижнему регистру. Пунктуацию можно оставить, так как она влияет на смысл предложения и на встречаемость.
Кроме этого можно избавиться от совсем коротких шуток, так как скорее всего это просто ответы на фразы. При делении текста на слова, используйте word_tokenize.

In [30]:
from tqdm import tqdm
tqdm.pandas()

In [21]:
from string import punctuation
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [32]:
def clean_text(text):
    '''
    Делит текст на слова и пунктуацию и приводит все к нижнему регистру
    :param text: строка
    :returns: список слов и знаков препинания
    '''
    text = text.lower()
    new_text = []
    for word in word_tokenize(text):
        new_text.append(word)
    return new_text

In [33]:
data['words'] = data['selftext'].progress_apply(clean_text)
data['lens'] = data['words'].progress_apply(len)
data = data[data.lens > 3]

100%|██████████| 494157/494157 [00:00<00:00, 828060.89it/s] 


In [ ]:
data = data.sample(100000)

In [34]:
words = data['words'].tolist()

In [35]:
words[:2]

[['my',
  'corona',
  'is',
  'covered',
  'with',
  'foreskin',
  'so',
  'it',
  'is',
  'not',
  'exposed',
  'to',
  'viruses',
  '.'],
 ['it', "'s", 'called', 'google', 'sheets', '.']]

## N-grams
Для начала попробуем создать самую простую модель, основанную на встречаемости н-граммы в корпусе.

In [36]:
from collections import defaultdict, Counter

In [ ]:
# добавляем токены начала и конца
BOS, EOS = '[bos]', '[eos]'

class NGramLanguageModel:
    def __init__(self, lines, n):
        assert n >= 1
        self.n = n
        counts = self.ngram_counts(lines, self.n)

        # перевести количества в вероятности
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        # <YOUR CODE HERE> #

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: строка запроса
        :returns: словарь с возможными продолжениями заданного префикса
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ BOS ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    @staticmethod
    def ngram_counts(lines: list, n: int) -> dict:
        '''
        Создаёт словарь, где каждому префиксу (n-1 слово) присваивается словарь,
        в котором ключи - слова, а значения - количество н-грамм в текстах
        :param lines: список списков
        :param n: количество слов в н-грамме
        :returns: словарь, в котором для каждого в префикса известно количество
        н-грамм с каждым словом
        '''
        # dictionary[(word1, word2)][word3] = count((word1, word2, word3))
        dictionary = defaultdict(Counter)
        for line in lines:
            new_line = [BOS] * (n-1) + line + [EOS]
            for i in range(n-1, len(new_line)):
                prefix = tuple(new_line[i-n+1:i])
                word = new_line[i]
                dictionary[prefix][word] += 1
        return dictionary

# Проверим работу функции ngram_counts
dummy_lines = sorted(words, key=len)[:100]
dummy_counts = NGramLanguageModel.ngram_counts(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[(BOS, BOS)]) == 66
assert dummy_counts[BOS, 'a']['melon'] == 1

# Проверим работу модели
dummy_lm = NGramLanguageModel(dummy_lines, n=3)
p_initial = dummy_lm.get_possible_next_tokens('')
assert p_initial.most_common(1)[0][0] == 'a'

1. Попробуем составить предложение используя жадный метод.

In [ ]:
def get_next_word(lm: NGramLanguageModel, prefix: str) -> str:
    '''
    :param lm: language model
    :param prefix: строка префикса
    :returns: следующее, наиболее вероятное, слово для данного префикса
    '''
    # <YOUR CODE HERE> #
    return # next word

In [ ]:
lm = NGramLanguageModel(words, n=3)
prefix = 'get'
repeat = 20
for _ in range(repeat):
    word = get_next_word(lm, prefix)
    prefix += ' ' + word
    if prefix.endswith(EOS):
        break

print(prefix)

In [ ]:
prefix = ''
word = get_next_word(lm, prefix)
while word != EOS:
    prefix += f' {word}'
    word = get_next_word(lm, prefix)

print(prefix + f'{word} ')

2. Выбор наиболее вероятного слова не показал хороших результатов. Тем более, он будет строить очень много похожих предложений, а нам хочется разнообразия. Давайте попробуем семплировать методом top-k: выбираем k наиболее встречаемых вариантов и из них выбираем один случайным образом.

In [ ]:
def get_next_word(lm: NGramLanguageModel, prefix: str, k:int) -> str:
    '''
    :param lm: language model
    :param prefix: строка префикса
    :param k: количество слов в top-k
    :returns: следующее, наиболее вероятное, слово для данного префикса
    '''
    # <YOUR CODE HERE> #
    return # next word

In [ ]:
lm = NGramLanguageModel(words, n=3)
prefix = 'get'
repeat = 20
for i in range(repeat):
    word = get_next_word(lm, prefix, 5)
    prefix += ' ' + word
    if prefix.endswith(EOS):
        break

print(prefix)

In [ ]:
prefix = ''
word = get_next_word(lm, '', 5)
while word != EOS:
    prefix += f'{word} '
    word = get_next_word(lm, prefix, 5)
print(prefix + f'{word} ')

3. Для сравнения можно сделать beam search. Напоминаем, что он на каждом шаге выбирает k наилучших вариантов - те, с которыми наибольшая вероятность всего предложения. Кроме этого надо не забыть, что мы переводим вероятности в логарифмы: таким образом считается сумма логарифмов вероятностей для каждой н-граммы, из которого состоит предложение.
Чтобы не пересчитывать каждый раз корпус, давайте будем считать вероятность последней н-граммы.

![picture](https://d2l.ai/_images/beam-search.svg)

\begin{align}
        \frac{1}{L^\alpha}\sum_{t'=1}^LlogP(y_{t'}|y_{t'-n}, ..., y_{t'-1})
    \end{align}



## Оценивание модели
Раз мы научились делать простую языковую модель, надо понять, насколько она хорошо описывает язык. Для оценивания этого используем перплексию.

$PPL(W) = e^{ln(P(W)^{-\frac{1}{N}})} =e^{{-\frac{1}{N}}ln(P(W))}$



1. Для начала нужно разделить данные на тренировочные и тестовые

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data['words'].sample(100000), train_size=0.8)

2. Давайте напишем функцию, которая будет считать перплексию для каждого отдельного предложения на основе n-грамм. Для каждой нграммы считается её вероятность. Нам её считать не надо, так как она уже записана в атрибуте _self.probs_ в нашем классе модели.

In [ ]:
def perplexity(model: NGramLanguageModel, text: list, n=1) -> float:
    """
    :param model: language model
    :param text: список н-грамм предложения
    :param n: количество слов в н-грамме
    :returns: значение перплексии одного предложения
    """
    result = 0
    length = 0
    # ngram: (word1, word2, word3) if n==3
    for ngram in text:
        if n == 1:
            prefix = ()
        else:
            prefix = ngram[:n-1]
        prob = model.probs[prefix][ngram[-1]]
        # <YOUR CODE HERE> #

    return

3. Теперь можем посчитать среднюю перплексию по всему датасету.

In [ ]:
n = 1
lm = NGramLanguageModel(X_train, n=n)
avg_ppl = 0
for sentence in X_test:
    test_sent = list(ngrams(sentence, n=n))
    value = perplexity(lm, test_sent, n=n)
    avg_ppl += value

avg_ppl /=  len(X_test)
# assert np.isclose(avg_ppl, 1857.90, atol=1e-1)
print(avg_ppl)

Попробуем теперь сравнить перплексию для моделей униграм, биграм и триграм. Почему результаты получились такие?

In [ ]:
lm = NGramLanguageModel(X_train, n=2)
all_ppls = []
avg_ppl = 0
for sentence in X_test:
    test_sent = list(ngrams(sentence, n=2))
    value = perplexity(lm, test_sent, 2)
    all_ppls.append((' '.join(sentence), value))
    avg_ppl += value
print(avg_ppl / len(X_test))

In [ ]:
s = sorted(all_ppls, key=lambda x: x[1])
print(s[0], s[-1])

In [ ]:
lm = NGramLanguageModel(X_train, n=3)
all_ppls = []
avg_ppl = 0
for sentence in X_test:
    test_sent = list(ngrams(sentence, n=3))
    value = perplexity(lm, test_sent, 3)
    all_ppls.append((' '.join(sentence), value))
    avg_ppl += value
print(avg_ppl / len(X_test))

In [ ]:
s = sorted(all_ppls, key=lambda x: x[1])
print(s[0], s[-1])

## Использование готовых инструментов
К счастью, нам всё писать необязательно. Необходимые функции и модули уже были написаны умными программистами. Давайте посмотрим на модуль nltk и на то, что он нам предлагает.

In [ ]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

n = 1
train_data, padded_vocab = padded_everygram_pipeline(n, X_train)
model = MLE(n)
model.fit(train_data, padded_vocab)

In [ ]:
test_data, _ = padded_everygram_pipeline(n, X_test)

avg_ppl = 0
length = 0
for i, test in enumerate(test_data):
    # <YOUR CODE HERE> #
    pass

Давайте повторим то же самое но с моделями на биграммах и триграммах. Сравните результаты. Почему они такие?

In [ ]:
# <YOUR CODE HERE> #

И давайте напоследок попробуем сгенерировать последовательность с помощью обученной модели:

In [ ]:
model.generate(20, text_seed=['he'])

--------------